# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf

We start by defining the $A^\sigma$ matrices:

In [ ]:
Θ = zeros(4,2,2,2)
Σ = zeros(2,2)
A = zeros(4,2,2,2)

# define Σ the singlet matrix
Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);

# define Θ the projection operator
for a in 1:2
    for b in 1:2
        for c in 1:2
            Θ[1,a,b,c] = (a+b+c == 3)
            Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
            Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
            Θ[4,a,b,c] = (a+b+c == 6)
        end
    end
end

# define A the total tensor
for σ in 1:4
    for b in 1:2
        A[σ,:,b,:] = Θ[σ,:,b,:]*Σ*sqrt(6/10)
    end
end

Local expectation values will be calculated using the following function:

In [ ]:
function expect_local(ψi::Array{Real}, op::Matrix{Real})
   Val = zeros(length(ψ))
    for i in 1:length(ψ)
        if length(size(ψ[1])[2:end])==1
            @tensor A1_Sop[sp, la] := ψ[i][s, la] * S[s, sp]
            @tensor V = A1_Sop[sp, la] * ψ[i][sp, la]
        elseif length(size(ψ[1])[2:end])==2
            @tensor A1_Sop[sp, la, lb] := ψ[i][s, la, lb] * S[s, sp]
            @tensor V = A1_Sop[sp, la, lb] * ψ[i][sp, la, lb]
        elseif length(size(ψ[1])[2:end])==3
            @tensor A1_Sop[sp, la, lb, lc] := ψ[i][s, la, lb, lc] * S[s, sp]
            @tensor V = A1_Sop[sp, la, lb, lc] * ψ[i][sp, la, lb, lc]
        else
            error("Invalid Tensor at ψ at index ", i)
        end
        Val[i] = V
    end 
    return Val
end

In [ ]:
# @tensor TR[σ1,σ2,σ3,σ4,σ5,σ6,b1,b2,b3,b4,b5,b6] := 
#     A[σ1,c6,b1,c1] * A[σ2,c1,b2,c2] * A[σ3,c2,b3,c3] * A[σ4,c3,b4,c4] * A[σ5,c4,b5,c5] * A[σ6,c5,b6,c6]

# @time @tensor Rop[σ1,σ2,σ3,σ4,σ5,σ6,b1,b2,b3,b4,b5,b6] := TR[σ1p,σ2,σ3,σ4,σ5,σ6,b1,b2,b3,b4,b5,b6] * S[σ1p, σ1];

# @tensor S1 = [σ1,σ2,σ3,σ4,σ5,σ6,b1,b2,b3,b4,b5,b6] * 
#              S[σ1,σ1p]

# S1 = 0
# c = 0
# for σ1 = 1:4^6
#     σ1arr = [parse(Int64,σ)+1 for σ in string(σ1-1, base=4, pad=6)]
#     println(c/8^12)
#     for σ2 = 1:4^6
#         σ2arr = [parse(Int64,σ)+1 for σ in string(σ2-1, base=4, pad=6)]
#         for b1 = 1:2^6
#             b1arr = [parse(Int64,b)+1 for b in string(b1-1, base=2, pad=6)]
#             for b2 = 1:2^6
#                 c += 1
#                 b2arr = [parse(Int64,b)+1 for b in string(b2-1, base=2, pad=6)]
                
#                 # indices = zip(σ1arr, σ2arr, b1arr, b2arr)
#                 TR1 = TR[σ1arr[1], σ1arr[2], σ1arr[3], σ1arr[4], σ1arr[5], σ1arr[6], b1arr[1], b1arr[2], b1arr[3], b1arr[4], b1arr[5], b1arr[6]]
#                 TR2 = TR[σ2arr[1], σ2arr[2], σ2arr[3], σ2arr[4], σ2arr[5], σ2arr[6], b2arr[1], b2arr[2], b2arr[3], b2arr[4], b2arr[5], b2arr[6]]
#                 S1 += TR1 * TR2 * S[σ1arr[1], σ2arr[1]]
#             end
#         end
#     end
# end
# @show S1

In order to view the expected structure of the wave function and the resulting spins, we will define the following function to plot a Honeycomb structure:

In [ ]:
function honeycomb_vert(n::Integer, a::Real)
    @assert a > 0 "Edge length must be positive"
    dx = a * sin(π/6)
    dy = a * cos(π/6)
    
    verts = [(0.0, 0.0)]
    for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
        for y in range(-dy*i, stop=dy*i, length=i+1)
            append!(verts, [(-x,y), (-x-a,y)])
        end
    end

    for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
        append!(verts, [(-n*(dx+a)-dx, y)])
    end
    
    return verts
end

We also need a function which gives for each point in the honeycomb the indices of its neighboors:

In [ ]:
function honeycomb_neighboors()

Now that this exists, we can define out honeycomb:

In [ ]:
for (x,y) in honeycomb_vert(5, 1)
    @show x
    @show y
end

In [ ]:
a(k) = 1/(k+sin(π/6)*(k+1))
plt = scatter()
for k in range(2, stop=2, step=1)
    scatter!(plt, honeycomb_vert(k, a(k)), aspect_ratio=:equal, label="", marker=:circle)
end
display(plt)

In [ ]:
ψ = [A[:,:,:,:] for i in 1:6]
# S = diagm([3/2, 1/2, -1/2, -3/2])